# Enduro World Series (EWS) web scraping and analysis

This notebook downloads and analyzes the results from the EWS from its inception to current, and we carry out analysis on these results.

First, we begin by downloading results and scraping the files from https://www.enduroworldseries.com/

In [17]:
import bs4
import requests
import typing_extensions
import pandas
import re
from PyPDF2 import PdfWriter, PdfReader


In [2]:
def download_page(url):
	headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'}

	req = requests.get(url, headers=headers)

	try:
		req.raise_for_status()
	except Exception as e:
		print(f'Downloading failed: {e}')
	
	return bs4.BeautifulSoup(req.text, "html.parser")

In [4]:
# test download of 2016 results

test_url = "https://www.enduroworldseries.com/races/1/2016/" 

ews_soup = download_page(test_url)

In [5]:
ews_soup

<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<title>Enduro World Series | EWS Race Series</title>
<meta content="" name="description"/>
<meta content="Enduro World Series Race Labs" name="author"/>
<meta content="Enduro World Series | EWS Race Series" property="og:title">
<meta content="List of all the events in the EWS calendar for the current year" property="og:description">
<meta content="https://admin.enduroworldseries.com/uploads/pagebanners/EWS_banner.png" property="og:image">
<!-- Mobile Meta -->
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<link href="/images/favicon.png" rel="shortcut icon"/>
<link href="https://fonts.googleapis.com/css?family=Open+Sans:400italic,700italic,400,700,300&amp;subset=latin,latin-ext" rel="stylesheet" type="text/css"/>
<link href="https://fonts.googleapis.com/css?family=PT+Serif" rel="stylesheet" type="text/css"/>
<link href="https://fonts.googleapis.com/css?family=Exo:100,200,300,400,500,500i,600,700,8

In [6]:
# for link in ews_soup.find_all('a'):
# 	url = link.get('href')
# 	# print(link.get('href'))
# 	if 'race' in url:
# 		print(url)
links = ews_soup.find_all('a', href=True)
links = [link.get('href') for link in links]

race_list_URL = [test_url[:-14]+str(link) for link in links if 'results' in link]

In [116]:
race_list_URL

['https://www.enduroworldseries.com/race/ews-round-1/ews-tweed-valley/202186/results/',
 'https://www.enduroworldseries.com/race/ews-round-2/ews-petzen-jamnica/202187/results/',
 'https://www.enduroworldseries.com/race/ews-round-3/ews-val-di-fassa-trentino/202188/results/',
 'https://www.enduroworldseries.com/race/ews-round-4/ews-whistler/202189/results/',
 'https://www.enduroworldseries.com/race/ews-round-5/ews-burke/202190/results/',
 'https://www.enduroworldseries.com/race/ews-round-6/ews-sugarloaf/202191/results/',
 'https://www.enduroworldseries.com/race/ews-round-7/ews-crans-montana/202192/results/',
 'https://www.enduroworldseries.com/race/ews-round-8/ews-loudenvielle/202193/results/',
 'https://www.enduroworldseries.com/race/ews-round-1/MontenbaikEnduroWorldSeries/201601/results/',
 'https://www.enduroworldseries.com/race/ews-round-2/CerroCatedralMontenbaikEnduroWorldSeriespresentedbyShimano/201602/results/',
 'https://www.enduroworldseries.com/race/ews-round-3/EmeraldEnduropre

16


In [7]:
test_race = download_page(race_list_URL[0])

In [8]:
test_pdf = test_race.find_all('a', href=True)
test_pdf = [(link.get('href'), link) for link in test_pdf]
test_pdf = [(pdf, pdf_link.get_text()) for pdf, pdf_link in test_pdf if '.pdf' in pdf]

In [9]:
test_pdf

[('https://admin.enduroworldseries.com/uploads/documents/races/1654354218.pdf',
  'Results Pro Stage'),
 ('https://admin.enduroworldseries.com/uploads/documents/races/1654368581.pdf',
  'Results EWS100'),
 ('https://admin.enduroworldseries.com/uploads/documents/races/1654368628.pdf',
  'Results EWS80'),
 ('https://admin.enduroworldseries.com/uploads/documents/races/1654469829.pdf',
  'Results EWS'),
 ('https://admin.enduroworldseries.com/uploads/documents/races/1654469868.pdf',
  'Rankings Series'),
 ('https://admin.enduroworldseries.com/uploads/documents/races/1654469962.pdf',
  'Rankings Teams'),
 ('https://admin.enduroworldseries.com/uploads/documents/races/1654535565.pdf',
  'Split S1'),
 ('https://admin.enduroworldseries.com/uploads/documents/races/1654535596.pdf',
  'Split S3'),
 ('https://admin.enduroworldseries.com/uploads/documents/races/1654535624.pdf',
  'Split S5'),
 ('https://admin.enduroworldseries.com/uploads/documents/races/1654535653.pdf',
  'Split S6')]

In [117]:
race_list_URL[0].split('/')

['https:',
 '',
 'www.enduroworldseries.com',
 'race',
 'ews-round-1',
 'ews-tweed-valley',
 '202186',
 'results',
 '']

In [37]:
req = requests.get('https://admin.enduroworldseries.com/uploads/documents/races/1584572245.pdf')
req.raise_for_status()

with open('raw_pdf/test3.pdf', 'wb') as f:
	f.write(req.content)

In [8]:
reader = PdfReader("raw_pdf/test3.pdf")

In [13]:
page1text = reader.pages[0].extract_text()
page2text = reader.pages[1].extract_text()

pages = [page.extract_text() for page in reader.pages]

In [10]:
reader.pages[0].extract_text()

'Pos Plate Name Nat Stage 1 Pos Stage 2 Pos Stage 3 Pos Stage 4 Pos Penalties Time Gap\n   Team EMBA Stage 5 Pos Stage 6 Pos Stage 7 PosCrankworx Rotorua Giant Toa Enduro\nRotorua, New Zealand\nMarch 26, 2017\nRESULTS OVERALL\nMEN \n1 39 MASTERS Wyn 0:03:16.65 2 0:05:37.35 5 0:04:54.78 17 0:06:48.68 6 0:37:31.72\n  GT Factory Racing NZL.MASW.1987 0:07:06.31 2 0:04:26.45 19 0:05:21.50 15\n2 62 WALKER Matt 0:03:22.02 8 0:05:31.33 2 0:04:39.74 4 0:06:53.18 12 0:37:35.40 +0:00:03.68\n NZL.WALM.1990 0:07:27.86 18 0:04:24.55 11 0:05:16.72 6\n3 63 MASTERS Eddie 0:03:32.63 330:05:29.15 10:04:27.36 1 0:06:53.40 13 0:37:37.95 +0:00:06.23\n NZL.MASE.1989 0:07:11.22 4 0:04:46.00 59 0:05:18.19 11\n4 13 HILL Sam 0:03:28.58 22 0:05:44.14 28 0:04:55.69 190:06:44.10 1 0:37:40.33 +0:00:08.61\n  Chain Reaction Cycles Mavic AUS.HILS.1985 0:07:19.73 7 0:04:17.81 20:05:10.28 1\n5 7 CALLAGHAN Greg 0:03:37.79 52 0:05:38.72 10 0:04:49.21 8 0:06:48.44 5 0:37:58.21 +0:00:26.49\n  Cube Action Team IRL.CALG.1991 0

In [11]:
page2 = page2text.split('\n')

In [78]:
# note the issues with the underlines 
pages[9].split('\n')

['Pos Plate Name Nat Stage 1 Pos Stage 2 Pos Stage 3 Pos Stage 4 Pos Penalties Time Gap',
 '   Team EMBA Stage 5 Pos Stage 6 Pos Stage 7 PosCrankworx Rotorua Giant Toa Enduro',
 'Rotorua, New Zealand',
 'March 26, 2017',
 'RESULTS OVERALL',
 'MEN ',
 '145 131 REIBELT Harrison 0:04:16.90 133 0:07:05.29 144 0:06:19.62 134 0:10:20.56 150 1:02:03.12 +0:24:31.40',
 ' AUS.REIH.1995 0:21:01.29 150 0:06:32.26 143 0:06:27.20 123',
 '146 224 GIBSON Thomas 0:05:38.16 155 0:07:36.05 150 0:07:46.32 153 0:09:57.75 149 1:02:10.78 +0:24:39.06',
 ' 0:17:14.34 146 0:06:36.00 146 0:07:22.16 142',
 '147 122 DA SILVA Goncalo 0:04:11.38 125 0:09:38.38 157 0:07:50.37 154 0:11:31.81 153 0:01:00.00 1:10:34.72 +0:33:03.00',
 ' POR.DA G.1987 0:19:28.41 147 0:08:13.09 148 0:08:41.28 148',
 '148 123 CAFFIN Paul 0:05:45.29 156 0:08:25.48 155 0:07:45.80 152 0:11:13.55 151 0:01:00.00 1:12:01.47 +0:34:29.75',
 ' AUS.CAFP.1978 0:22:05.00 151 0:07:46.01 147 0:08:00.34 145',
 'DNF 53 CHAPMAN Rupert 0:17:39.98 159 0:05:43

In [73]:
position_plate_name_regex = re.compile(r'\d+\s\d+\s\w+\s\w+')
stage_position_regex = re.compile(r'\d:\d\d:\d\d\.\d\d \d+') # recognizes the a stage with its position
stage_time_regex = re.compile(r'(\d:\d\d:\d\d\.\d\d)') # recognizes each stage time (assumes all stages less than 10 hours)
gap_regex = re.compile(r'\+\d:\d\d:\d\d\.\d\d')
penalty_regex = re.compile(r'\d:\d\d:\d\d\.\d\d\s+\d:\d\d:\d\d\.\d\d') # penalty values occur between 


In [87]:
line = '4 13 HILL Sam 0:03:28.58 22 0:05:44.14 28 0:04:55.69 190:06:44.10 1 0:37:40.33 +0:00:08.61'
line2 = '  Chain Reaction Cycles Mavic AUS.HILS.1985 0:07:19.73 7 0:04:17.81 20:05:10.28 1'

line = '147 122 DA SILVA Goncalo 0:04:11.38 125 0:09:38.38 157 0:07:50.37 154 0:11:31.81 153 0:01:00.00 1:10:34.72 +0:33:03.00'
line2 = ' POR.DA G.1987 0:19:28.41 147 0:08:13.09 148 0:08:41.28 148'

fix = stage_time_regex.sub(r' \1', line+line2) # adds space before each stage time - used to fix issue with formatting of underlines
fix = fix.replace('+ ', '+') # removes space before gap time

In [88]:
fix

'147 122 DA SILVA Goncalo  0:04:11.38 125  0:09:38.38 157  0:07:50.37 154  0:11:31.81 153  0:01:00.00  1:10:34.72 +0:33:03.00 POR.DA G.1987  0:19:28.41 147  0:08:13.09 148  0:08:41.28 148'

In [89]:
ppnr = position_plate_name_regex.search(fix)
spr = stage_position_regex.findall(fix)

In [90]:
pr = penalty_regex.search(fix)

In [91]:
pr.group()

'0:01:00.00  1:10:34.72'

In [92]:
ppnr.group()

'147 122 DA SILVA'

In [93]:
spr

['0:04:11.38 125',
 '0:09:38.38 157',
 '0:07:50.37 154',
 '0:11:31.81 153',
 '0:19:28.41 147',
 '0:08:13.09 148',
 '0:08:41.28 148']